In [1]:
import pandas as pd

df = pd.read_csv("data/diamonds.csv")

In [2]:
df.head(100)

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
95,0.70,Good,E,VS2,57.5,58.0,2759,5.85,5.90,3.38
96,0.70,Good,F,VS1,59.4,62.0,2759,5.71,5.76,3.40
97,0.96,Fair,F,SI2,66.3,62.0,2759,6.27,5.95,4.07
98,0.73,Very Good,E,SI1,61.6,59.0,2760,5.77,5.78,3.56


In [3]:
df = df[df['color'] == "D" ] 

In [4]:
len(df)
# uuups das sind ja ziemlich viele....

6775

In [5]:
# Preis auf Basis carat schätzen

X = df[["carat"]].values
Y = df[["price"]].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0, test_size = 0.25)

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.8089443227790487


In [6]:
# Preis auf Basis Gewicht Länge Breite Höhe schätzen

X = df[["carat", "x", "y","z"]].values
Y = df[["price"]].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0, test_size = 0.25)

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.8183307162955448


In [7]:
# Preis auf Basis Länge Breite Höhe schätzen

X = df[["x", "y","z"]].values
Y = df[["price"]].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 0, test_size = 0.25)

from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)

print(model.score(X_test, y_test))

0.752497681920969


In [8]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree = 2, include_bias = False)
pf.fit(X_train) #wäre hier strenggenommen nicht nötig, aber andere Prozesse verlangen das später

PolynomialFeatures(include_bias=False)

In [9]:
X_train
#Ein Array aus Tupeln...

array([[6.  , 5.93, 3.46],
       [4.15, 4.18, 2.58],
       [5.2 , 5.26, 3.21],
       ...,
       [6.68, 6.75, 4.24],
       [4.46, 4.43, 2.7 ],
       [8.12, 7.99, 5.08]])

In [10]:
pf.transform(X_train)
#erzeugt  jetzt 5 Zahlen pro Example und zwar: x1, x2, x3, x1*x1, X1*x2, x1*x3, ....   

array([[ 6.    ,  5.93  ,  3.46  , ..., 35.1649, 20.5178, 11.9716],
       [ 4.15  ,  4.18  ,  2.58  , ..., 17.4724, 10.7844,  6.6564],
       [ 5.2   ,  5.26  ,  3.21  , ..., 27.6676, 16.8846, 10.3041],
       ...,
       [ 6.68  ,  6.75  ,  4.24  , ..., 45.5625, 28.62  , 17.9776],
       [ 4.46  ,  4.43  ,  2.7   , ..., 19.6249, 11.961 ,  7.29  ],
       [ 8.12  ,  7.99  ,  5.08  , ..., 63.8401, 40.5892, 25.8064]])

In [11]:
pf.transform(X_train)[0]
# also: c, x, y, z, c*c, c*x, c*y, .... usw. usw.

array([ 6.    ,  5.93  ,  3.46  , 36.    , 35.58  , 20.76  , 35.1649,
       20.5178, 11.9716])

In [12]:
#hier der Bauplan wo welche Potenzen sind.
pf.powers_

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1],
       [2, 0, 0],
       [1, 1, 0],
       [1, 0, 1],
       [0, 2, 0],
       [0, 1, 1],
       [0, 0, 2]], dtype=int64)

In [13]:
X_train_transformed = pf.transform(X_train)
X_test_transformed = pf.transform(X_test)
#X_train_transformed = pf.transform(X_train)[:, [0, 1, 2, 3, 4]]
#X_test_transformed = pf.transform(X_test)[:, [0, 1, 2, 3, 4]]
# Durch manuelles trial-and-error (Herumprobieren), kann man herausfinden, was hier wirklich
# ausschlaggebend ist, um auf R2 möglichst hoch zu bekommen.
#X_train_transformed = pf.transform(X_train)[:, [0, 2]]
#X_test_transformed = pf.transform(X_test)[:, [0, 2]]


In [14]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_transformed, y_train)

print(model.score(X_test_transformed, y_test))

0.8304138693834557


In [15]:
model.coef_

array([[ 14947.87040842, -19707.81402271,  -5849.13334263,
        -21924.44945414,  34473.85843816,  10889.08666973,
        -18486.10249669,  10363.78797104, -16173.22137199]])